In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Desktop\\apex\\project\\Predicting_Bitcoin_Prices_Using_ANN\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Desktop\\apex\\project\\Predicting_Bitcoin_Prices_Using_ANN'

In [70]:
from dataclasses import dataclass
from pathlib import Path
from typing import List


@dataclass(frozen=True)
class ANNTrainingConfig:
    root_dir: Path
    data_file: Path
    model_save_dir: Path
    # project_name: str
    updated_base_model_path: Path
    trained_model_path: Path

    # Data parameters
    sample_size: int
    target_column: str

    # Training parameters
    epochs_tuning: int
    epochs_final: int
    batch_size: int
    learning_rate: float

    # Hyperparameter tuning ranges
    max_trials: int
    num_layers_range: List[int]
    units_range: List[int]
    dropout_range: List[float]


In [71]:
from ANNClassifier.constants import *
from ANNClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [72]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])

    
    def get_ann_training_config(self) -> ANNTrainingConfig:
        training = self.config['training']
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        create_directories([training['root_dir']])

        training_config = ANNTrainingConfig(
            root_dir=Path(training['root_dir']),
            data_file=Path(training['data_file']),
            model_save_dir=Path(training['model_save_dir']),
            trained_model_path=Path(training.trained_model_path),
            # updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            updated_base_model_path=Path(prepare_base_model.base_model_path),
            # project_name=training['project_name'],

            sample_size=params['SAMPLE_SIZE'],
            target_column=params['TARGET_COLUMN'],

            epochs_tuning=params['EPOCHS_TUNING'],
            epochs_final=params['EPOCHS_FINAL'],
            batch_size=params['BATCH_SIZE'],
            learning_rate=params['LEARNING_RATE'],

            max_trials=params['MAX_TRIALS'],
            num_layers_range=params['NUM_LAYERS_RANGE'],
            units_range=params['UNITS_RANGE'],
            dropout_range=params['DROPOUT_RANGE']
        )

        return training_config

In [73]:
import os
import urllib.request as request
import tensorflow as tf
import time
import pickle
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [74]:
class ANNModelTrainer:
    def __init__(self, config: ANNTrainingConfig):
        self.config = config

    # def get_base_model(self):
    #     # This loads the model saved after initial compilation
    #     # self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
    #     self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
        self.config.updated_base_model_path
    )


    def load_data(self):
        """Load preprocessed and saved training data"""
        with open("artifacts/data_preparation/train_val_data.pkl", "rb") as f:
            data = pickle.load(f)

        self.X_train = data["X_train"]
        self.X_val = data["X_val"]
        self.y_train = data["y_train"]
        self.y_val = data["y_val"]

    def load_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-5,
            verbose=1
        )

        history = self.model.fit(
            self.X_train,
            self.y_train,
            validation_data=(self.X_val, self.y_val),
            epochs=self.config.epochs_final,
            batch_size=self.config.batch_size,
            callbacks=[reduce_lr],
            verbose=1
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

        print("✅ Model training complete. Model saved to:", self.config.trained_model_path)

In [75]:
# from ANNClassifier.components.model_trainer import ANNModelTrainer

try:
    config = ConfigurationManager()
    training_config = config.get_ann_training_config()
    training = ANNModelTrainer(config=training_config)
    training.get_base_model()

    training.load_data()       # load preprocessed data from .pkl
    training.load_model() 
    # training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

Epoch 1/100
219/219 [==============================] - 2s 9ms/step - loss: 0.0036 - mean_absolute_error: 0.0332 - val_loss: 4.7746e-05 - val_mean_absolute_error: 0.0063 - lr: 0.0010
Epoch 2/100
219/219 [==============================] - 2s 8ms/step - loss: 9.5035e-04 - mean_absolute_error: 0.0210 - val_loss: 0.0030 - val_mean_absolute_error: 0.0495 - lr: 0.0010
Epoch 3/100
219/219 [==============================] - 1s 6ms/step - loss: 6.8985e-04 - mean_absolute_error: 0.0174 - val_loss: 1.1262e-04 - val_mean_absolute_error: 0.0086 - lr: 0.0010
Epoch 4/100
219/219 [==============================] - 2s 7ms/step - loss: 5.6197e-04 - mean_absolute_error: 0.0157 - val_loss: 0.0049 - val_mean_absolute_error: 0.0624 - lr: 0.0010
Epoch 5/100
219/219 [==============================] - 2s 7ms/step - loss: 5.9349e-04 - mean_absolute_error: 0.0156 - val_loss: 0.0011 - val_mean_absolute_error: 0.0272 - lr: 0.0010
Epoch 6/100
219/219 [==============================] - ETA: 0s - loss: 4.8418e-04 - me